In [ ]:
import yfinance as yf
import pandas as pd
import ta
import time
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer



api_key = "cul25nhr01qqav2uqppgcul25nhr01qqav2uqpq0"
url = f"https://finnhub.io/api/v1/stock/symbol?exchange=US&token={api_key}"
response = requests.get(url)
tickers_data = response.json()

tickers = [ticker["symbol"] for ticker in tickers_data]


def coletar_dados(ticker):
    dados = yf.download(ticker, period="30d", interval="1d")

    # Garantir que "Close" seja uma série 1D
    dados["Close"] = dados["Close"].astype(float).squeeze()

    # Calcular a Média Móvel Simples (SMA) de 10 e 20 dias
    dados["SMA_10"] = dados["Close"].rolling(window=10).mean()
    dados["SMA_20"] = dados["Close"].rolling(window=20).mean()

    # Calcular o RSI (Relative Strength Index) de 14 dias
    rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
    dados["RSI"] = rsi_indicator.rsi()

    # Calcular as Bandas de Bollinger (20 dias)
    bollinger = ta.volatility.BollingerBands(close=dados["Close"].squeeze(), window=20)
    dados["Bollinger_High"] = bollinger.bollinger_hband()
    dados["Bollinger_Low"] = bollinger.bollinger_lband()
    return dados

def gerar_recomendacao(df):
    if len(df) < 20:
        return "Dados insuficientes para análise."

    ult_linha = df.iloc[-1]  # Última linha corretamente indexada
    recomendacoes = []

    # 🔹 Verificando se "RSI" existe e não é NaN antes da análise
    if "RSI" in df.columns:
        rsi_valor = ult_linha["RSI"].values[0] if isinstance(ult_linha["RSI"], pd.Series) else ult_linha["RSI"]
        print("Última linha:", ult_linha)  # Debugging: print the last row
        print("Valor do RSI:", rsi_valor)  # Debugging: print the RSI value
        
        if isinstance(rsi_valor, (float, int)) and not pd.isna(rsi_valor):  # Verificando se o valor não é NaN
            if rsi_valor < 30:
                recomendacoes.append("RSI indica COMPRA (ativo sobrevendido).")
            elif rsi_valor > 70:
                recomendacoes.append("RSI indica VENDA (ativo sobrecomprado).")
            
    # 🔹 Verificando cruzamento de médias móveis
    if len(df) > 1:
        prev_linha = df.iloc[-2]
        sma_10_prev = prev_linha["SMA_10"].values[0] if isinstance(prev_linha["SMA_10"], pd.Series) else prev_linha["SMA_10"]
        sma_20_prev = prev_linha["SMA_20"].values[0] if isinstance(prev_linha["SMA_20"], pd.Series) else prev_linha["SMA_20"]
        sma_10_ult = ult_linha["SMA_10"].values[0] if isinstance(ult_linha["SMA_10"], pd.Series) else ult_linha["SMA_10"]
        sma_20_ult = ult_linha["SMA_20"].values[0] if isinstance(ult_linha["SMA_20"], pd.Series) else ult_linha["SMA_20"]

        if sma_10_prev < sma_20_prev and sma_10_ult > sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica COMPRA.")
        elif sma_10_prev > sma_20_prev and sma_10_ult < sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica VENDA.")

    # 🔹 Analisando Bandas de Bollinger
    if "Close" in df.columns and "Bollinger_Low" in df.columns and "Bollinger_High" in df.columns:
        close_valor = ult_linha["Close"].values[0] if isinstance(ult_linha["Close"], pd.Series) else ult_linha["Close"]
        bollinger_low = ult_linha["Bollinger_Low"].values[0] if isinstance(ult_linha["Bollinger_Low"], pd.Series) else ult_linha["Bollinger_Low"]
        bollinger_high = ult_linha["Bollinger_High"].values[0] if isinstance(ult_linha["Bollinger_High"], pd.Series) else ult_linha["Bollinger_High"]

        if close_valor < bollinger_low:
            recomendacoes.append("Banda de Bollinger indica COMPRA (preço abaixo da banda inferior).")
        elif close_valor > bollinger_high:
            recomendacoes.append("Banda de Bollinger indica VENDA (preço acima da banda superior).")
    
    return "\n".join(recomendacoes) if recomendacoes else "Nenhuma recomendação clara no momento."


def coletar_noticias(ticker):
    url = f"https://newsapi.org/v2/everything?q={ticker}&apiKey=1aaf7f3923f543fe89ae1dd8688d0a1c"
    resposta = requests.get(url)
    dados = resposta.json()
    
    artigos = [
        (artigo["title"] + " " + (artigo["description"] if artigo["description"] else ""))
        for artigo in dados["articles"]
    ]
    return artigos[:5]  # Pegamos apenas as 5 notícias mais recentes


nltk.download("vader_lexicon")

def analisar_sentimento_vader(noticias):
    sia = SentimentIntensityAnalyzer()
    scores = [sia.polarity_scores(noticia)["compound"] for noticia in noticias]

    media_sentimento = sum(scores) / len(scores) if scores else 0

    if media_sentimento > 0.05:
        return "Positivo"
    elif media_sentimento < -0.05:
        return "Negativo"
    else:
        return "Neutro"
print("\n🔄 Atualizando análise...")

for ticker in tickers:
    print(f"Analisando {ticker}...")
    dados = coletar_dados(ticker)
    noticias = coletar_noticias(ticker)
    sentimento = analisar_sentimento_vader(noticias)
    recomendacao = gerar_recomendacao(dados)
    
    print(f"📊 Recomendação para {ticker}: {recomendacao}")
    print(f"Sentimento das notícias: {sentimento}")
    print("\n" + "-"*50 + "\n")
    time.sleep(1)

In [ ]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import requests
import matplotlib.pyplot as plt
import time

# 🔹 Obter todos os tickers disponíveis via API do Finnhub
api_key = "cul25nhr01qqav2uqppgcul25nhr01qqav2uqpq0"
url = f"https://finnhub.io/api/v1/stock/symbol?exchange=US&token={api_key}"
response = requests.get(url)
tickers_data = response.json()

tickers = [ticker["symbol"] for ticker in tickers_data]

# 🔹 Função para buscar dados e prever para cada ação
def prever_para_acao(ticker, model, scaler):
    print(f"🔍 Buscando dados para {ticker}...")

    try:
        # Baixar os dados do Yahoo Finance
        dados = yf.download(ticker, period="2y", interval="1d")
        fechamento = dados["Close"].values.reshape(-1, 1)

        # 🔹 Normalizar os dados
        fechamento_normalizado = scaler.fit_transform(fechamento)

        # 🔹 Criar sequências para a LSTM
        def criar_sequencias(data, seq_length=60):
            X = []
            for i in range(len(data) - seq_length):
                X.append(data[i:i+seq_length])
            return np.array(X)

        seq_length = 60
        X = criar_sequencias(fechamento_normalizado, seq_length)

        # 🔹 Prever para o futuro
        ultimos_60_dias = fechamento_normalizado[-seq_length:].reshape(1, seq_length, 1)
        previsao_normalizada = model.predict(ultimos_60_dias)
        previsao_real = scaler.inverse_transform(previsao_normalizada)[0][0]

        return previsao_real

    except Exception as e:
        print(f"⚠️ Erro ao processar {ticker}: {e}")
        return None


def criar_sequencias(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])  # O próximo valor após a sequência

    return np.array(X), np.array(y)
# 🔹 Criar e treinar o modelo LSTM
def criar_modelo_lstm(seq_length=60):
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(100, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 🔹 Carregar o modelo ou treinar se não existir
modelo_path = "modelo_lstm.h5"
scaler = MinMaxScaler(feature_range=(0, 1))

if os.path.exists(modelo_path):
    print(f"✅ Carregando modelo salvo...")
    model = load_model(modelo_path)
else:
    print(f"🚀 Treinando novo modelo...")
    model = criar_modelo_lstm()

    # Baixar os dados históricos (2 anos)
    primeiro_ticker = tickers[0]
    dados = yf.download(primeiro_ticker, period="2y", interval="1d")
    fechamento = dados["Close"].values.reshape(-1, 1)
    fechamento_normalizado = scaler.fit_transform(fechamento)

    # Definir tamanhos dos conjuntos (1.5 anos treino, 3 meses teste, 3 meses validação)
    total_dias = len(fechamento_normalizado)
    treino_size = int(total_dias * 0.75)   # 75% para treino (~1 ano e meio)
    teste_size = int(total_dias * 0.125)   # 12.5% para teste (~3 meses)

    # Criar conjuntos de treino, teste e validação
    treino_data = fechamento_normalizado[:treino_size]
    teste_data = fechamento_normalizado[treino_size:treino_size + teste_size]
    valid_data = fechamento_normalizado[treino_size + teste_size:]

    # Criar sequências (X = entradas, y = valores reais esperados)
    seq_length = 60
    X_treino, y_treino = criar_sequencias(treino_data, seq_length)
    X_teste, y_teste = criar_sequencias(teste_data, seq_length)
    X_valid, y_valid = criar_sequencias(valid_data, seq_length)

    # Treinar modelo apenas com os dados de treino
    model.fit(X_treino, y_treino, epochs=50, batch_size=16, validation_data=(X_valid, y_valid))

    # Avaliar o modelo nos dados de teste
    loss = model.evaluate(X_teste, y_teste)
    print(f"📊 Loss no conjunto de teste: {loss}")

    # Salvar o modelo treinado
    model.save(modelo_path)
    print(f"✅ Modelo salvo como '{modelo_path}'!")


# Exibir lista de tickers disponíveis
print("📈 Ações disponíveis para previsão:")
for i, ticker in enumerate(tickers):
    print(f"{i + 1}. {ticker}")

# Solicitar escolha do usuário
while True:
    ticker_escolhido = input("Digite o código da ação que deseja prever: ").upper()
    if ticker_escolhido in tickers:
        break
    else:
        print("❌ Código inválido! Digite um ticker da lista.")

# Tentar baixar os dados com re-tentativa
tentativas = 3
print("⏳ Aguardando antes de tentar novamente...")
time.sleep(5)
for tentativa in range(tentativas):
    try:
        print(f"🔄 Buscando dados para {ticker_escolhido}... (Tentativa {tentativa+1}/{tentativas})")
        dados = yf.download(ticker_escolhido, period="2y", interval="1d")
        
        # Verificar se os dados foram baixados corretamente
        if dados.empty:
            raise ValueError("⚠️ Nenhum dado retornado. O ticker pode estar incorreto ou não ter dados suficientes.")

        fechamento = dados["Close"].values.reshape(-1, 1)
        fechamento_normalizado = scaler.fit_transform(fechamento)

        # Gerar previsão
        previsao = prever_para_acao(ticker_escolhido, model, scaler)

        # Salvar a previsão em um arquivo CSV
        if previsao is not None:
            previsao_df = pd.DataFrame([[ticker_escolhido, previsao]], columns=["Ticker", "Previsão"])
            previsao_df.to_csv(f"previsao_{ticker_escolhido}.csv", index=False)
            print(f"✅ Previsão salva em 'previsao_{ticker_escolhido}.csv'!")
        else:
            print("❌ Não foi possível gerar a previsão para esta ação.")
        break  # Se conseguiu baixar, sair do loop
    except Exception as e:
        print(f"⚠️ Erro ao processar {ticker_escolhido}: {e}")
        if tentativa < tentativas - 1:
            print("⏳ Aguardando antes de tentar novamente...")
            time.sleep(10)  # Esperar antes de tentar novamente
        else:
            print("❌ Falha após múltiplas tentativas. Tente novamente mais tarde.")